In [2]:
import pandas as pd

# Cargar el archivo CSV
file_path = 'input/hanna_stories_annotations.csv'
output_csv = 'output/worker_rankings_dataset.csv'
df = pd.read_csv(file_path)

# Crear la columna 'story_id' combinando 'Story ID' y 'Model'
df['story_id'] = df['Story ID'].astype(str) + '_' + df['Model']

# Definir las columnas de puntuación a considerar
score_columns = ['Relevance', 'Coherence', 'Empathy', 'Surprise', 'Engagement', 'Complexity']

# Calcular la suma de puntuaciones para cada historia
df['total_score'] = df[score_columns].sum(axis=1)

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación total de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking: se utiliza dense para que en caso de empates se asigne la misma posición
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total (para conservar la magnitud)
    group['ranking_value'] = group['total_score']
    # Calcular el ranking value normalizado por usuario (min-max)
    min_score = group['ranking_value'].min()
    max_score = group['ranking_value'].max()
    if max_score - min_score != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_score) / (max_score - min_score)
    else:
        group['ranking_value_norm'] = 0.0
    return group

# Aplicar el ranking para cada trabajador (Worker ID)
df_ranked = df.groupby('Worker ID').apply(assign_ranking).reset_index(drop=True)

# Seleccionar las columnas deseadas y renombrar 'Worker ID' a 'worker_id'
df_output = df_ranked[['Worker ID', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']]\
    .rename(columns={'Worker ID': 'worker_id'})

# Exportar el resultado a CSV
df_output.to_csv(output_csv, index=False)

print(f"CSV file created: {output_csv}")

CSV file created: output/worker_rankings_dataset.csv


/tmp/ipykernel_421293/693865860.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df.groupby('Worker ID').apply(assign_ranking).reset_index(drop=True)
